In [1]:
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
import keras
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import math
import nltk

In [2]:
from google.colab import files

uploaded = files.upload()

Saving dataset.csv to dataset.csv


In [7]:
import io
df = pd.read_csv(io.BytesIO(uploaded['dataset.csv']))

In [8]:
data = df.copy()
def remove_tags(string):
    removelist = ""
    result = re.sub('','',string)
    result = re.sub('https://.*','',result)
    result = re.sub(r'[^w'+removelist+']', ' ',result)
    result = result.lower()
    return result
data['review']=data['review'].apply(lambda cw : remove_tags(cw))

In [9]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
data['review'] = data['review'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [11]:
import nltk
nltk.download('wordnet')
w_tokenizer = nltk.tokenize.WhitespaceTokenizer()
lemmatizer = nltk.stem.WordNetLemmatizer()
def lemmatize_text(text):
    st = ""
    for w in w_tokenizer.tokenize(text):
        st = st + lemmatizer.lemmatize(w) + " "
    return st
data['review'] = data.review.apply(lemmatize_text)

[nltk_data] Downloading package wordnet to /root/nltk_data...


In [12]:
s = 0.0
for i in data['review']:
    word_list = i.split()
    s = s + len(word_list)
print("Average length of each review : ",s/data.shape[0])
pos = 0
for i in range(data.shape[0]):
    if data.iloc[i]['sentiment'] == 'positive':
        pos = pos + 1
neg = data.shape[0]-pos
print("Percentage of reviews with positive sentiment is "+str(pos/data.shape[0]*100)+"%")
print("Percentage of reviews with negative sentiment is "+str(neg/data.shape[0]*100)+"%")

Average length of each review :  18.714
Percentage of reviews with positive sentiment is 50.0%
Percentage of reviews with negative sentiment is 50.0%


In [13]:
reviews = data['review'].values
labels = data['sentiment'].values
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)

In [14]:
train_sentences, test_sentences, train_labels, test_labels = train_test_split(reviews, encoded_labels, stratify = encoded_labels)

In [15]:
vocab_size = 3000
oov_tok = ''
embedding_dim = 100
max_length = 200
padding_type='post'
trunc_type='post'

tokenizer = Tokenizer(num_words = vocab_size, oov_token=oov_tok)
tokenizer.fit_on_texts(train_sentences)
word_index = tokenizer.word_index

train_sequences = tokenizer.texts_to_sequences(train_sentences)
train_padded = pad_sequences(train_sequences, padding='post', maxlen=max_length)

test_sequences = tokenizer.texts_to_sequences(test_sentences)
test_padded = pad_sequences(test_sequences, padding='post', maxlen=max_length)

In [16]:
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim, input_length=max_length),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])

model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])


In [19]:
num_epochs = 30
history = model.fit(train_padded, train_labels,
                    epochs=num_epochs, verbose=1,
                    validation_split=0.1)

Epoch 1/30
1055/1055 [==============================] - 20s 19ms/step - loss: 0.6928 - accuracy: 0.5117 - val_loss: 0.6929 - val_accuracy: 0.4989
Epoch 2/30
1055/1055 [==============================] - 21s 19ms/step - loss: 0.6927 - accuracy: 0.5128 - val_loss: 0.6923 - val_accuracy: 0.5107
Epoch 3/30
1055/1055 [==============================] - 19s 18ms/step - loss: 0.6927 - accuracy: 0.5126 - val_loss: 0.6922 - val_accuracy: 0.5107
Epoch 4/30
1055/1055 [==============================] - 21s 19ms/step - loss: 0.6927 - accuracy: 0.5130 - val_loss: 0.6925 - val_accuracy: 0.5067
Epoch 5/30
1055/1055 [==============================] - 19s 18ms/step - loss: 0.6926 - accuracy: 0.5153 - val_loss: 0.6925 - val_accuracy: 0.5107
Epoch 6/30
1055/1055 [==============================] - 19s 18ms/step - loss: 0.6926 - accuracy: 0.5146 - val_loss: 0.6920 - val_accuracy: 0.5091
Epoch 7/30
1055/1055 [==============================] - 21s 20ms/step - loss: 0.6926 - accuracy: 0.5153 - val_loss: 0.6926 -

In [20]:
prediction = model.predict(test_padded)

pred_labels = []
for i in prediction:
    if i >= 0.5:
        pred_labels.append(1)
    else:
        pred_labels.append(0)
print("Accuracy of prediction on test set : ", accuracy_score(test_labels,pred_labels))

391/391 [==============================] - 3s 7ms/step
Accuracy of prediction on test set :  0.51976
